# Preprocessing

We will process our data set and prepare it for modelling based on our EDA findings.

In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
census_income_learn = dataiku.Dataset("census_income_learn")
census_income_learn_df = census_income_learn.get_dataframe()
census_income_test = dataiku.Dataset("census_income_test")
census_income_test_df = census_income_test.get_dataframe()


# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

processed_learn_df = ... # Compute a Pandas dataframe to write into processed_learn
processed_test_df = ... # Compute a Pandas dataframe to write into processed_test


# Write recipe outputs
processed_learn = dataiku.Dataset("processed_learn")
processed_learn.write_with_schema(processed_learn_df)
processed_test = dataiku.Dataset("processed_test")
processed_test.write_with_schema(processed_test_df)